# Parsing through Database Tables

## Dependencies / Import Data

In [22]:
print(eagleview_table.columns)

Index(['Claim #', 'Job #', 'Square Feet', 'Ridges', 'Hips', 'Valleys', 'Rakes',
       'Eaves'],
      dtype='object')


In [18]:
# import data_prep
import numpy as np

import pandas as pd

############### PROJECT TABLE DATA ####################
project_table_data = "./data/database_tables/project_table.csv"
project_table = pd.read_csv(
    project_table_data, 
    dtype={'Claim #': str,'Job #': str,'Branch':str,'Claim Status':str},
    parse_dates=[
        'Rep Agreement Signed', 'Rep Claim Collected','FTA Scope Completed',
        'FTA Scope Rejected', 'BC Estimate Completed','OB Scope Completed',
        'Sup Job Submitted', 'BC Approved for Production', 'OB Order Built',
        'GM Order Processed', 'PA Permit Applied', 'PA Permit Processed',
        'PA OA Processed', 'PA OA Invoiced', 'PA Notify of Delivery',
        'PA Notify of Start', 'Delivery Date', 'Roof Start',
        'Roof End', 'GM Approved for Inspection', 'GM Change Order Date',
        'GM Labor Adjustment Date', 'RA Inspection Requested', 'RA Inspection Processed', 
        'Rep COC Collected', 'SA Job Docs Uploaded', 'BC Project Invoiced','BC Project Closed'])

############### PROJECT INFORMATION TABLE DATA ###############
project_info_data = "./data/database_tables/project_info_table.csv"

info_table = pd.read_csv(project_info_data, dtype={
    'Claim #':str, 'Job #':str, 'Branch':str, 'City':str, 'Building Department':str,
    'Permit Req?':str, 'Supplier Name':str, 'Crew':str, 'Insurance Company':str,
    'Multi-rejected':str, 'Scope Rejections':str, 'Change Orders':str,'Labor Adjustments':str})

# due to manual 'city' field, city is incorrectly spelled or off on capitalization
info_table['City'] = info_table['City'].str.upper()

#################### WORKFLOW TABLE DATA ####################
workflow_table_data = "./data/database_tables/workflow_table.csv"
workflow_table = pd.read_csv(workflow_table_data)

######################### EAGLEVIEW DATA #########################
eagleview_table_data = "./data/raw_datasets/[TVA] EagleView Analysis.csv"
eagleview_table = pd.read_csv(eagleview_table_data)

# converting the squares measurement to roofing SQs
eagleview_table['Square Feet'] = eagleview_table['Square Feet'] / 100

# removing all duplcates
eagleview_table = eagleview_table.drop_duplicates(subset='Claim #', keep='first')


# using a dict to point the 'selected_branch' function to the correct column
pipeline_dict = {  
    'Rep Claims': {'start':'Rep Agreement Signed', 'end':'Rep Claim Collected', 'other':'none',
                   'project_table':['Claim #', 'Job #', 'Branch', 'Claim Status', 'Rep Agreement Signed', 'Rep Claim Collected'],
                   'workflow_table':['Claim #', 'Rep Collecting Claim'],
                   'info_table':['Claim #', 'City'],
                   'eagleview_table':['Claim #', 'Square Feet'],
                   'clean_table':['Claim #', 'Job #', 'Branch', 'City', 'Rep Agreement Signed', 'Rep Claim Collected', 'Rep Collecting Claim', 'Square Feet']
                  },
    'FTA Scopes': {'start':'Rep Claim Collected', 'end':'FTA Scope Completed', 'other':'none',
                   'project_table':['Claim #','Job #','Branch','Claim Status','Rep Claim Collected','FTA Scope Completed','FTA Scope Rejected',],
                   'workflow_table':['Claim #','FTA Completing Scope','Days in Pipeline',],
                   'info_table':['Claim #','Job #','Branch','City','Building Department','Scope Rejections',],
                   'eagleview_table':['Claim #','Job #','Square Feet',],
                   'clean_table':[],
                  }, 
    'BC Estimates': {'start':'FTA Scope Completed', 'end':'BC Estimate Completed', 'other':'none',
                   'project_table':['Claim #','Job #','Branch','Claim Status','FTA Scope Completed','BC Estimate Completed',],
                   'workflow_table':['Claim #','BC Completing Estimate','Days in Pipeline',],
                   'info_table':['Claim #','Job #','Branch','City','Building Department',],
                   'eagleview_table':['Claim #','Job #','Square Feet',],
                   'clean_table':[],
                    },
    'OB Scopes': {'start':'BC Estimate Completed', 'end':'OB Scope Completed', 'other':'FTA Scope Rejected',
                   'project_table':['Claim #','Job #','Branch','Claim Status','BC Estimate Completed','OB Scope Completed',],
                   'workflow_table':['Claim #','OB Completing Scope','Days in Pipeline',],
                   'info_table':['Claim #','Job #','Branch','City','Building Department','Scope Rejections',],
                   'eagleview_table':['Claim #','Job #','Square Feet',],
                   'clean_table':[],
                    },
    'Sup Submittals': {'start':'OB Scope Completed', 'end':'Sup Job Submitted', 'other':'Job #',
                   'project_table':['Claim #','Job #','Branch','Claim Status','OB Scope Completed','Sup Job Submitted',],
                   'workflow_table':['Claim #','Rep Collecting Claim','Sup Submitting Job','Days in Pipeline',],
                   'info_table':['Claim #','Job #','Branch','City','Insurance Company',],
                   'eagleview_table':['Claim #','Job #','Square Feet',],
                   'clean_table':[],
                    },
    'BC Approvals': {'start':'Sup Job Submitted', 'end':'BC Approved for Production','other':'none',
                   'project_table':['Claim #','Job #','Branch','Claim Status','Sup Job Submitted','BC Approved for Production',],
                   'workflow_table':['Claim #','BC Approving Job','Days in Pipeline',],
                   'info_table':['Claim #','Job #','Branch','City','Building Department','Insurance Company',],
                   'eagleview_table':['Claim #','Job #','Square Feet',],
                   'clean_table':[],
                    },    
    'OB Orders Built': {'start':'BC Approved for Production', 'end':'OB Order Built','other':'none',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'GM Orders Processed': {'start':'OB Order Built', 'end':'GM Order Processed','other':'none',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'PA Permits Applied': {'start':'GM Order Processed', 'end':'PA Permit Applied','other':'Permit Req?',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'PA Permits Processed':{'start':'PA Permit Applied', 'end':'PA Permit Processed','other':'Permit Req?',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'PA OA Processed':{'start':'GM Order Processed', 'end':'PA OA Processed','other':'none',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'PA OA Invoiced':{'start':'PA OA Processed', 'end':'PA OA Invoiced','other':'none',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'PA Notify of Delivery':{'start':'Delivery Date', 'end':'PA Notify of Delivery','other':'none',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'PA Notify of Start':{'start':'Roof Start', 'end':'PA Notify of Start','other':'none',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    }, 
    
    
    'GM Approved for Inspection':{'start':'Roof End', 'end':'GM Approved for Inspection','other':'Permit Req?',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'RA Inspection Requested':{'start':'GM Approved for Inspection', 'end':'RA Inspection Requested','other':'Permit Req?',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'RA Inspection Processed':{'start':'RA Inspection Requested', 'end':'RA Inspection Processed','other':'Permit Req?',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'Rep COC Collected':{'start':'Roof End', 'end':'Rep COC Collected','other':'none',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'SA Job Docs Uploaded':{'start':'Rep COC Collected', 'end':'SA Job Docs Uploaded','other':'none',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    
    
    'BC Project Invoiced':{'start':'SA Job Docs Uploaded', 'end':'BC Project Invoiced','other':'none',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
    'BC Project Closed':{'start':'BC Project Invoiced', 'end':'BC Project Closed','other':'none',
                   'project_table':[],
                   'workflow_table':[],
                   'info_table':[],
                   'eagleview_table':[],
                   'clean_table':[],
                    },
}

In [11]:
def create_pipeline(pipeline):

    pipeline_info = pipeline_dict.get(pipeline)
    
    project_table_info = pipeline_info['project_table']
    workflow_table_info = pipeline_info['workflow_table']
    info_table_info = pipeline_info['info_table']
    eagleview_table_info = pipeline_info['eagleview_table']
#     clean_table_list = pipeline_info['clean_table']
    
    # prodviding only revelant information from project table
    parsed_project_table = project_table[project_table_info]
    
    # providing only revelant information from workflow table
    parsed_workflow_table = workflow_table[workflow_table_info]
    
    # providing only revelant information from info table
    parsed_info_table = info_table[info_table_info]
    
    # providing only revelant information from eagleview table
    parsed_eagleview_table = eagleview_table[eagleview_table_info]
    
    # merging all parsed tables together
    merged_table = parsed_project_table.merge(
        parsed_workflow_table, on='Claim #', how='left').merge(
        parsed_info_table, on='Claim #', how='left').merge(
        parsed_eagleview_table, on='Claim #', how='left')
    
#     merged_table = merged_table[clean_table_list]

    return merged_table.head()

rep_claim_pipeline('Rep Claims')

,Claim #,Job #,Branch,City,Rep Agreement Signed,Rep Claim Collected,Rep Collecting Claim,Square Feet
0,01-001-706959,1939638,FCO,LOVELAND,2019-08-11,2019-08-12,1.0,22.13
1,H3J6971001H,1939538,COS,COLORADO SPRINGS,2019-08-08,2019-08-10,2.0,18.51
2,PRO2019146280,1939528,KCI,SAVANNAH,2019-08-07,2019-08-07,0.0,16.37
3,ICK8570,1939465,DEN,WESTMINSTER,2019-08-06,2019-08-06,0.0,25.95
4,01001761567,1939447,KCI,SAINT JOSEPH,2019-08-05,2019-08-08,3.0,51.37


In [ ]:
def create_pipeline():

    # prodviding only revelant information from project table
    parsed_project_table = project_table[['Claim #', 'Job #', 'Branch', 'Claim Status', 'Rep Agreement Signed', 'Rep Claim Collected']].copy()
    
    # providing only revelant information from workflow table
    parsed_workflow_table = workflow_table[['Claim #', 'Rep Collecting Claim']]
    
    # providing only revelant information from info table
    parsed_info_table = info_table[['Claim #', 'City']]
    
    # providing only revelant information from eagleview table
    parsed_eagleview_table = eagleview_table[['Claim #', 'Square Feet']]
    
    # merging all parsed tables together
    merged_table = parsed_project_table.merge(
        parsed_workflow_table, on='Claim #', how='left').merge(
        parsed_info_table, on='Claim #', how='left').merge(
        parsed_eagleview_table, on='Claim #', how='left')
    
    merged_table = merged_table[['Claim #', 'Job #', 'Branch', 'City', 'Rep Agreement Signed', 
                                 'Rep Claim Collected', 'Rep Collecting Claim', 'Square Feet']]

    return merged_table.head()

rep_claim_pipeline()

## Creating Pipeline Class

In [26]:
# combining 'permit' info from 'info table' with pipeline data
info_table_data = info_table[['Job #', 'Permit Req?']]
filtered_info_table_data = info_table_data.loc[(info_table_data['Job #'].isnull() == False ),:]
pipeline_df = project_table.merge(filtered_info_table_data, how='left', on='Job #')


class Branch_Pipeline:
    
    def __init__(self, branch, pipeline):
        self.branch = branch
        self.pipeline = pipeline

    
    def create_teammate_pipeline(self):
        
        # assigning the dict to a variable to get dict values
        pipeline_dates = pipeline_dict.get(self.pipeline)
        
        start = pipeline_dates['start']
        end = pipeline_dates['end']
        other = pipeline_dates['other']

        # filtering by specific branch or ALL
        if self.branch == 'All':
            branch_data = pipeline_df

        else:
            branch_data = pipeline_df.loc[(project_table['Branch'] == self.branch), :]

        # filtering the pipelines without an 'other' condition
        if other == 'none':
            
            pipeline_data = branch_data.loc[(
                branch_data[start].isnull() == False )&(
                branch_data[end].isnull() == True),:]
            
        # filtering the pipelines dependent on permit requirements
        elif other == 'Permit Req?':
            
            pipeline_data = branch_data.loc[(
                branch_data[start].isnull() == False )&(
                branch_data[end].isnull() == True)&(
                branch_data[other] == 'Y'),:]
            
            
        else:
            # pipelines with a specific 'other' value
            pipeline_data = branch_data.loc[(
                branch_data[start].isnull() == False )&(
                branch_data[end].isnull() == True)&(
                branch_data[other].isnull() == True),:]
        
        # reproducing the user input
        print(f'Branch being Analyzed: {self.branch}')
        print(f'Pipeline Chosen: {self.pipeline}')
        
        # storing the 'end' date (+1) index as a variable to slice the df
        end_col_index = pipeline_data.columns.get_loc(end) + 1
        
        # slicing the df at the last revelant column
        teammate_pipeline_data = pipeline_data.iloc[:, 0:end_col_index]
        
        print(f'Current Pipeline: {len(teammate_pipeline_data)} records.')
        return teammate_pipeline_data

## User Pipeline Analysis

In [24]:
def run_pipeline_analysis():
    
    # providing the branch options
    
    print('''
    #########################################################################################

                                        POSSIBLE BRANCHES

                                ALL | DEN | COS | FCO | KCI | OMA

    #########################################################################################
    ''')
    
    # storing the 'branch' response as a variable
    branch_response = input('Choose your Branch: ')
    
    # providing the pipeline options
    
    print('''
    
    #################################################################
    
                            POSSIBLE PIPELINES

    Rep Claims | FTA Scopes| BC Estimates | OB Scopes | Sup Submittals
    
    BC Approvals | OB Orders Built | GM Orders Processed | PA Permits Applied
    
    PA Permits Processed | PA OA Processed | PA OA Invoiced | PA Notify of Delivery
    
    PA Notify of Start | GM Approved for Inspection | RA Inspection Requested
    
    RA Inspection Processed | Rep COC Collected | SA Job Docs Uploaded 
    
    BC Project Invoiced | BC Project Closed
    
    ######################################################################
    
    ''')
    
    
    # storing the 'pipeline' response as a variable
    pipeline_response = input("Generate which Pipeline?: ")
    
    # creates 
    analysis = Branch_Pipeline(branch_response, pipeline_response)
    
    return analysis.create_teammate_pipeline()

In [37]:
# combining 'permit' info from 'info table' with pipeline data
info_table_data = info_table[['Job #', 'Permit Req?']]
filtered_info_table_data = info_table_data.loc[(info_table_data['Job #'].isnull() == False ),:]
pipeline_df = project_table.merge(filtered_info_table_data, how='left', on='Job #')


class Branch_Workflow:
    
    def __init__(self, branch, pipeline):
        self.branch = branch
        self.pipeline = pipeline

    
    def create_teammate_workflow(self):
        
        # assigning the dict to a variable to get dict values
        pipeline_dates = pipeline_dict.get(self.pipeline)
        
        start = pipeline_dates['start']
        end = pipeline_dates['end']
        other = pipeline_dates['other']

        # filtering by specific branch or ALL
        if self.branch == 'All':
            branch_data = pipeline_df

        else:
            branch_data = pipeline_df.loc[(project_table['Branch'] == self.branch), :]

        # filtering the pipelines without an 'other' condition
        if other == 'none':
            
            workflow_data = branch_data.loc[(
                branch_data[start].isnull() == False )&(
                branch_data[end].isnull() == False),:]
            
        # filtering the pipelines dependent on permit requirements
        elif other == 'Permit Req?':
            
            workflow_data = branch_data.loc[(
                branch_data[start].isnull() == False )&(
                branch_data[end].isnull() == False)&(
                branch_data[other] == 'Y'),:]
            
            
        else:
            # pipelines with a specific 'other' value
            workflow_data = branch_data.loc[(
                branch_data[start].isnull() == False )&(
                branch_data[end].isnull() == False)&(
                branch_data[other].isnull() == True),:]
                       
        
        
        # reproducing the user input
        print(f'Branch being Analyzed: {self.branch}')
        print(f'Pipeline Chosen: {self.pipeline}')
        
        # storing the 'end' date (+1) index as a variable to slice the df
        end_col_index = workflow_data.columns.get_loc(end) + 1
        
        # slicing the df at the last revelant column
        teammate_workflow_data = workflow_data.iloc[:, 0:end_col_index]
        
        return teammate_workflow_data

In [38]:
def run_workflow_analysis():
    
    # providing the branch options
    
    print('''
    #########################################################################################

                                        POSSIBLE BRANCHES

                                ALL | DEN | COS | FCO | KCI | OMA

    #########################################################################################
    ''')
    
    # storing the 'branch' response as a variable
    branch_response = input('Choose your Branch: ')
    
    # providing the pipeline options
    
    print('''
    
    #################################################################
    
                            POSSIBLE PIPELINES

    Rep Claims | FTA Scopes| BC Estimates | OB Scopes | Sup Submittals
    
    BC Approvals | OB Orders Built | GM Orders Processed | PA Permits Applied
    
    PA Permits Processed | PA OA Processed | PA OA Invoiced | PA Notify of Delivery
    
    PA Notify of Start | GM Approved for Inspection | RA Inspection Requested
    
    RA Inspection Processed | Rep COC Collected | SA Job Docs Uploaded 
    
    BC Project Invoiced | BC Project Closed
    
    ######################################################################
    
    ''')
    
    
    # storing the 'pipeline' response as a variable
    workflow_response = input("Generate which Workflow?: ")
    
    # creates 
    analysis = Branch_Workflow(branch_response, workflow_response)
    
    return analysis.create_teammate_workflow()

In [39]:
tester = run_workflow_analysis()
tester.head()


    #########################################################################################

                                        POSSIBLE BRANCHES

                                ALL | DEN | COS | FCO | KCI | OMA

    #########################################################################################
    
Choose your Branch: KCI

    
    #################################################################
    
                            POSSIBLE PIPELINES

    Rep Claims | FTA Scopes| BC Estimates | OB Scopes | Sup Submittals
    
    BC Approvals | OB Orders Built | GM Orders Processed | PA Permits Applied
    
    PA Permits Processed | PA OA Processed | PA OA Invoiced | PA Notify of Delivery
    
    PA Notify of Start | GM Approved for Inspection | RA Inspection Requested
    
    RA Inspection Processed | Rep COC Collected | SA Job Docs Uploaded 
    
    BC Project Invoiced | BC Project Closed
    
    #################################################################

,Claim #,Job #,Branch,Claim Status,Rep Agreement Signed,Rep Claim Collected,FTA Scope Completed,FTA Scope Rejected,BC Estimate Completed,OB Scope Completed,...,Roof End,GM Approved for Inspection,GM Change Order Date,GM Labor Adjustment Date,RA Inspection Requested,RA Inspection Processed,Rep COC Collected,SA Job Docs Uploaded,BC Project Invoiced,BC Project Closed
613,IFR1654,1937201,KCI,NaN,2019-06-17,2019-06-25,2019-06-30,2019-07-08,2019-07-10,2019-07-10,...,2019-07-26,2019-07-30,NaT,NaT,NaT,NaT,2019-07-25,2019-08-08,2019-08-08,2019-08-15
621,010001610666,1937174,KCI,NaN,2019-06-17,2019-06-17,2019-06-24,2019-06-25,2019-06-26,2019-06-26,...,2019-07-18,2019-07-18,NaT,2019-07-23,NaT,NaT,2019-07-18,2019-07-31,2019-08-01,2019-08-12
625,001824322-412,1937164,KCI,NaN,2019-06-17,2019-06-17,2019-06-23,NaT,2019-06-24,2019-06-24,...,2019-07-11,2019-07-11,NaT,2019-07-15,NaT,NaT,2019-07-11,2019-07-24,2019-07-24,2019-07-24
695,01001602093,1937005,KCI,NaN,2019-06-11,2019-06-16,2019-06-21,NaT,2019-06-25,2019-06-24,...,2019-07-16,2019-07-16,NaT,2019-07-23,2019-07-23,2019-07-25,2019-07-16,2019-07-29,2019-08-05,2019-08-05
725,9227178-12,1936943,KCI,NaN,2019-06-11,2019-06-13,2019-06-19,2019-06-19,2019-06-20,2019-06-20,...,2019-07-14,2019-07-14,NaT,2019-07-15,NaT,NaT,2019-07-31,2019-08-01,2019-08-01,2019-08-02
